# RUL estimation Nasa Randomized dataset

In [ ]:
import numpy as np
import pandas as pd
import scipy.io
import math
import os
import ntpath
import sys
import logging
import time
import sys
import random

from importlib import reload
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import LSTM, Embedding, RepeatVector, TimeDistributed, Masking, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback


IS_COLAB = False
IS_TRAINING = True
RESULT_NAME = ""

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = "/content/drive/My Drive/CEM-Data-Experiment/cem-data-experiment/"
else:
    data_path = "../../"

sys.path.append(data_path)
from data_processing.nasa_random_data import NasaRandomizedData
from data_processing.prepare_rul_data import RulHandler

### Config logging

In [ ]:
reload(logging)
logging.basicConfig(format='%(asctime)s [%(levelname)s]: %(message)s', level=logging.DEBUG, datefmt='%Y/%m/%d %H:%M:%S')

# Load Data

In [ ]:
train_names = [
        'Battery_Uniform_Distribution_Variable_Charge_Room_Temp_DataSet_2Post/data/Matlab/RW1',
        'Battery_Uniform_Distribution_Variable_Charge_Room_Temp_DataSet_2Post/data/Matlab/RW2',
        'Battery_Uniform_Distribution_Variable_Charge_Room_Temp_DataSet_2Post/data/Matlab/RW7',

        #'Battery_Uniform_Distribution_Discharge_Room_Temp_DataSet_2Post/data/Matlab/RW3',
        'Battery_Uniform_Distribution_Discharge_Room_Temp_DataSet_2Post/data/Matlab/RW4',
        'Battery_Uniform_Distribution_Discharge_Room_Temp_DataSet_2Post/data/Matlab/RW5',

        #'Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/Matlab/RW9',
        #'Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/Matlab/RW10',
        #'Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/Matlab/RW11',

        'RW_Skewed_Low_Room_Temp_DataSet_2Post/data/Matlab/RW13',
        'RW_Skewed_Low_Room_Temp_DataSet_2Post/data/Matlab/RW14',
        'RW_Skewed_Low_Room_Temp_DataSet_2Post/data/Matlab/RW15',

        'RW_Skewed_High_Room_Temp_DataSet_2Post/data/Matlab/RW17',
        'RW_Skewed_High_Room_Temp_DataSet_2Post/data/Matlab/RW18',
        'RW_Skewed_High_Room_Temp_DataSet_2Post/data/Matlab/RW19',

        'RW_Skewed_Low_40C_DataSet_2Post/data/Matlab/RW21',
        'RW_Skewed_Low_40C_DataSet_2Post/data/Matlab/RW22',
        'RW_Skewed_Low_40C_DataSet_2Post/data/Matlab/RW23',

        'RW_Skewed_High_40C_DataSet_2Post/data/Matlab/RW25',
        'RW_Skewed_High_40C_DataSet_2Post/data/Matlab/RW26',
        'RW_Skewed_High_40C_DataSet_2Post/data/Matlab/RW27',
]

test_names = [
        'Battery_Uniform_Distribution_Variable_Charge_Room_Temp_DataSet_2Post/data/Matlab/RW8',
        #'Battery_Uniform_Distribution_Discharge_Room_Temp_DataSet_2Post/data/Matlab/RW6',
        #'Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/Matlab/RW12',
        'RW_Skewed_Low_Room_Temp_DataSet_2Post/data/Matlab/RW16',
        'RW_Skewed_High_Room_Temp_DataSet_2Post/data/Matlab/RW20',
        'RW_Skewed_Low_40C_DataSet_2Post/data/Matlab/RW24',
        'RW_Skewed_High_40C_DataSet_2Post/data/Matlab/RW28',
]

In [ ]:
nasa_data_handler = NasaRandomizedData(data_path)

rul_handler = RulHandler()

## Data preparation

In [ ]:
CAPACITY_THRESHOLDS = None
NOMINAL_CAPACITY = 2.2
N_CYCLE = 500
WARMUP_TRAIN = 15
WARMUP_TEST = 30

(train_x, train_y_soh, test_x, test_y_soh,
  train_battery_range, test_battery_range,
  time_train, time_test, current_train, current_test) = nasa_data_handler.get_discharge_whole_cycle_future(train_names, test_names)

train_y = rul_handler.prepare_y_future(train_names, train_battery_range, train_y_soh, current_train, time_train, CAPACITY_THRESHOLDS, capacity=NOMINAL_CAPACITY)
del globals()["current_train"]
del globals()["time_train"]
test_y = rul_handler.prepare_y_future(test_names, test_battery_range, test_y_soh, current_test, time_test, CAPACITY_THRESHOLDS, capacity=NOMINAL_CAPACITY)
del globals()["current_test"]
del globals()["time_test"]
train_x, test_x = rul_handler.compress_cycle(train_x, test_x)

x_norm = rul_handler.Normalization()
train_x, test_x = x_norm.normalize(train_x, test_x)

train_x = rul_handler.battery_life_to_time_series(train_x, N_CYCLE, train_battery_range)
test_x = rul_handler.battery_life_to_time_series(test_x, N_CYCLE, test_battery_range)

train_x, train_y, train_battery_range, train_y_soh = rul_handler.delete_initial(train_x, train_y, train_battery_range, train_y_soh, WARMUP_TRAIN)
test_x, test_y, test_battery_range, test_y_soh = rul_handler.delete_initial(test_x, test_y, test_battery_range, test_y_soh, WARMUP_TEST)

train_x, train_y, train_battery_range, train_y_soh = rul_handler.limit_zeros(train_x, train_y, train_battery_range, train_y_soh)
test_x, test_y, test_battery_range, test_y_soh = rul_handler.limit_zeros(test_x, test_y, test_battery_range, test_y_soh)

# first one is SOH, we keep only RUL
train_y = train_y[:,1]
test_y = test_y[:,1]

### Y normalization

In [ ]:
y_norm = rul_handler.Normalization()
train_y, test_y = y_norm.normalize(train_y, test_y)

# Model training

In [ ]:
if IS_TRAINING:
    EXPERIMENT = "lstm_rul_nasa_randomized"

    experiment_name = time.strftime("%Y-%m-%d-%H-%M-%S") + '_' + EXPERIMENT
    print(experiment_name)

    # Model definition

    opt = tf.keras.optimizers.Adam(lr=0.000003)

    model = Sequential()
    model.add(Masking(input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(LSTM(128, activation='selu',
                    return_sequences=True,
                    kernel_regularizer=regularizers.l2(0.0002)))
    model.add(LSTM(64, activation='selu', return_sequences=False,
                    kernel_regularizer=regularizers.l2(0.0002)))
    model.add(Dense(64, activation='selu', kernel_regularizer=regularizers.l2(0.0002)))
    model.add(Dense(32, activation='selu', kernel_regularizer=regularizers.l2(0.0002)))
    model.add(Dense(1, activation='linear'))
    model.summary()

    model.compile(optimizer=opt, loss='huber', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])

In [ ]:
if IS_TRAINING:
    history = model.fit(train_x, train_y,
                                epochs=500, 
                                batch_size=32, 
                                verbose=1,
                                validation_split=0
                               )

In [ ]:
if IS_TRAINING:
    model.save(data_path + 'results/trained_model/%s.h5' % experiment_name)

    hist_df = pd.DataFrame(history.history)
    hist_csv_file = data_path + 'results/trained_model/%s_history.csv' % experiment_name
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)
    history = history.history

In [ ]:
if not IS_TRAINING:
    history = pd.read_csv(data_path + 'results/trained_model/%s_history.csv' % RESULT_NAME)
    model = keras.models.load_model(data_path + 'results/trained_model/%s.h5' % RESULT_NAME)
    model.summary()

In [ ]:
if not IS_TRAINING:
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(history)

### Testing

In [ ]:
results = model.evaluate(test_x, test_y, return_dict = True)
print(results)
max_rmse = 0
for index in range(test_x.shape[0]):
    result = model.evaluate(np.array([test_x[index, :, :]]), np.array([test_y[index]]), return_dict = True, verbose=0)
    max_rmse = max(max_rmse, result['rmse'])
print("Max rmse: {}".format(max_rmse))

# Results Visualization

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history['loss'],
                    mode='lines', name='train'))
fig.update_layout(title='Loss trend',
                  xaxis_title='epoch',
                  yaxis_title='loss',
                  width=1400,
                  height=600)
fig.show()

In [ ]:
train_predictions = model.predict(train_x)

train_y = y_norm.denormalize(train_y)
train_predictions = y_norm.denormalize(train_predictions)

In [ ]:
a = 0
for b in train_battery_range:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_y_soh[a:b], y=train_predictions[a:b,0],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(x=train_y_soh[a:b], y=train_y[a:b],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on training',
                    xaxis_title='SoH Capacity',
                    yaxis_title='Remaining Ah until EOL',
                    xaxis={'autorange':'reversed'},
                    width=1400,
                    height=600)
    fig.show()
    a = b

In [ ]:
a = 0
for b in train_battery_range:
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=train_predictions[a:b,0],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=train_y[a:b],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on training',
                    xaxis_title='Cycle',
                    yaxis_title='Remaining Ah until EOL',
                    width=1400,
                    height=600)
    fig.show()
    a = b

In [ ]:
test_predictions = model.predict(test_x)

test_y = y_norm.denormalize(test_y)
test_predictions = y_norm.denormalize(test_predictions)

In [ ]:
a = 0
for b in test_battery_range:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test_y_soh[a:b], y=test_predictions[a:b,0],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(x = test_y_soh[a:b], y=test_y[a:b],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on testing',
                    xaxis_title='SoH Capacity',
                    yaxis_title='Remaining Ah until EOL',
                    xaxis={'autorange':'reversed'},
                    width=1400,
                    height=600)
    fig.show()
    a = b

In [ ]:
a = 0
for b in test_battery_range:
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=test_predictions[a:b, 0],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=test_y[a:b],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on testing',
                    xaxis_title='Cycle',
                    yaxis_title='Remaining Ah until EOL',
                    width=1400,
                    height=600)
    fig.show()
    a = b